## Imports, libraries



In [1]:
# install relevant libraries
!pip install git+https://github.com/deepset-ai/FARM.git
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install gensim==3.4.0

  Cloning https://github.com/deepset-ai/FARM.git to /tmp/pip-req-build-4lc1zr4r
  Running command git clone -q https://github.com/deepset-ai/FARM.git /tmp/pip-req-build-4lc1zr4r
     |████████████████████████████████| 43 kB 920 kB/s 
     |████████████████████████████████| 14.1 MB 1.3 MB/s 
     |████████████████████████████████| 2.5 MB 24.7 MB/s 
     |████████████████████████████████| 85 kB 3.6 MB/s 
     |████████████████████████████████| 327 kB 48.5 MB/s 
     |████████████████████████████████| 2.5 MB 22.5 MB/s 
     |████████████████████████████████| 79 kB 6.3 MB/s 
     |████████████████████████████████| 1.1 MB 22.1 MB/s 
     |████████████████████████████████| 345 kB 26.5 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
     |████████████████████████████████| 85 kB 3.3 MB/s 
     |████████████████████████████████| 165 kB 50.7 MB/s 
     |████████████████████████████████| 45 kB 2.5 MB/s 
     |████████████████████████████████| 93 kB 1.2 MB/s 
     |██████████████████

In [2]:
# imports
# doing it twice removes errors ;) ;)
for i in range(2):
    try:
        import collections
        from tqdm import tqdm
        from glob import glob
        from farm.utils import initialize_device_settings
        from haystack.document_store.faiss import FAISSDocumentStore
        from haystack.preprocessor import PreProcessor
        from haystack.retriever.dense import DensePassageRetriever
        from haystack import Pipeline
        #from haystack.eval import EvalReader, EvalRetriever>
        import numbers
        import random
        from time import perf_counter
        import csv
        from matplotlib import pyplot
        import uuid
        from sklearn.feature_extraction.text import TfidfVectorizer
        from sklearn.metrics.pairwise import cosine_similarity
        from gensim.summarization.bm25 import BM25
        import numpy as np
        import pandas as pd
        from sklearn.utils import shuffle
        import json
    except:
        print('something wrong')

/opt/conda/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


something wrong


In [ ]:
!nvidia-smi

## Evaluation


In [3]:
# init model names and paths
mdl_1 = [{'model':'LaDPR'},
         {'qr_mdl': 'diarsabri/LaDPR-query-encoder'},
         {'ps_mdl': 'diarsabri/LaDPR-context-encoder'}]
mdl_2 = [{'model':'DPR'},
        {'qr_mdl': "facebook/dpr-question_encoder-single-nq-base"},
        {'ps_mdl': "facebook/dpr-ctx_encoder-single-nq-base"}]

In [4]:
# test init variables
#fil = '/content/drive/MyDrive/Colab Notebooks/nq_dev_subset_v2.json'
fil = '../input/mexdpr/MLQA_ALL_70k.json'
#fil = '../input/mexdpr/MLQA_ALL_20k.json'

file_name = fil.split('mexdpr/')[-1][:-5]
model = mdl_1
k = 20

In [5]:
# doc store init
doc_store = FAISSDocumentStore(faiss_index_factory_str="Flat")
device, n_gpu = initialize_device_settings(use_cuda=True)

doc_index = "docs_dpr"
label_index = "lbls_dpr"

In [6]:
# doc store adding data
doc_store.delete_documents(index=doc_index)
doc_store.delete_documents(index=label_index)

doc_store.add_eval_data(
    filename=fil,
    doc_index=doc_index,
    label_index=label_index
    #,max_docs=30
)

In [7]:
len(doc_store.get_all_documents(index=doc_index))

33915

In [8]:
# choosing model and initializing DPR, and updating doc store embeddings
qr_mdl = model[1]['qr_mdl']
ps_mdl = model[2]['ps_mdl']

dpr = DensePassageRetriever(document_store=doc_store,
                                    query_embedding_model=qr_mdl,
                                    passage_embedding_model=ps_mdl,
                                    use_gpu=True,
                                    embed_title=True,
                                    max_seq_len_query=64,
                                    max_seq_len_passage=256,
                                    batch_size=16)

doc_store.update_embeddings(dpr, index=doc_index)

Downloading:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/604 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/855 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

...

Documents Processed:  59%|█████▉    | 20000/33915 [03:39<02:24, 96.19 docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Documents Processed:  88%|████████▊ | 30000/33915 [05:11<00:40, 96.62 docs/s]

Create embeddings:   0%|          | 0/3920 [00:00<?, ? Docs/s]

Documents Processed: 40000 docs [05:52, 113.61 docs/s]


In [9]:
# creating dictionaries for documents
docs = doc_store.get_all_documents(index=doc_index)

# unique doc set ids
doc_set = {}

# give doc sets unique ids
for d in docs:
  id = str(uuid.uuid4())
  ds = d.meta['doc_set']

  if not ds in doc_set:
    doc_set[ds] = id
    d.meta['doc_set_id'] = id
  else:
    curid = doc_set[ds]
    d.meta['doc_set_id'] = curid

# create dict for doc set - all docs
ds_d = {}
for d in docs:
  did = d.id
  ds = d.meta['doc_set']
  dsid = d.meta['doc_set_id']

  if dsid in ds_d:
    ds_d[dsid].append(did)
  else:
    ds_d[dsid] = [did]

In [10]:
# return document set id, number of related docs, related doc ids, correct doc ids from given doc id
def get_doc_set_from_doc(did):
  dsid, dids = [], []
  for d in did: # if question has multiple answers, need to loop
    
    ds = doc_store.get_document_by_id(id=d,index=doc_index).meta['doc_set']
    dsid.append(doc_set[ds])
    dids.append(ds_d[dsid[-1]])

  dids_flat = [item for sublist in dids for item in sublist]
  dic = {'dsid':dsid,
        'n_docs':len(dids_flat),
        'dids':dids_flat,
        'cor_doc':did}
  return dic

In [11]:
labels = doc_store.get_all_labels_aggregated(index=label_index)

question_label_dict = {}
for l in labels:
  q = l.question
  a_id = l.multiple_document_ids
  dic = get_doc_set_from_doc(a_id)

  question_label_dict[q] = dic

In [12]:
def compute_f1(gold_toks, pred_toks):
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  # return {'precision':precision,
  #         'recall':recall,
  #         'f1':f1}
  return float(f1)

In [13]:
# tf-idf
def tf_idf_topk(query, all_docs, k):
  texts = [d.text for d in all_docs]
  ids = [d.id for d in all_docs]
  
  # Initialize a vectorizer that removes English stop words
  vectorizer = TfidfVectorizer()

  # Create a corpus of query and documents and convert to TFIDF vectors
  query_and_docs = [query] + texts
  matrix = vectorizer.fit_transform(query_and_docs)

  # Holds our cosine similarity scores
  # The first vector is our query text, so compute the similarity of our query against all document vectors
  scores = [cosine_similarity(matrix[0], matrix[i])[0][0] for i in range(1, len(query_and_docs))]

  # Sort list of scores and return the top k highest scoring documents
  sorted_list = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
  top_doc_indices = [x[0] for x in sorted_list[:k]]
  top_docs = [ids[x] for x in top_doc_indices]
  
  return top_docs

In [14]:
# bm25
def bm25_topk(query, all_docs, k):
  texts = [d.text for d in all_docs]
  ids = [d.id for d in all_docs]

  bm25 = BM25(texts)
  average_idf = sum(float(val) for val in bm25.idf.values()) / len(bm25.idf)
  scores = bm25.get_scores(query, average_idf)

  best_docs = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
  doc_ids = [ids[i] for i in best_docs]
  return doc_ids

In [15]:
# evaluates model, either ladpr/dpr/tfidf/bm25
# returns metrics on recall, map, mrr, f1 (only really applicable on k = 1)
def eval(top_ks,func):
  all_metrics = []

  for k in top_ks:
    predictions = []

    #timed_retrieve = dpr.timing(dpr.retrieve, "retrieve_time")

    correct_retrievals = 0
    summed_avg_precision = 0.0
    summed_reciprocal_rank = 0.0
    avg_f1 = 0.0
    number_of_questions = len(question_label_dict)
    all_docs = doc_store.get_all_documents(index=doc_index)

    for question, qdict in tqdm(question_label_dict.items()):
      
      if func == 'LaDPR' or func == 'DPR':
        timed_retrieve = dpr.timing(dpr.retrieve, "retrieve_time")
            
        retrieved_docs = [d.id for d in timed_retrieve(question, top_k=k, index=doc_index)]
      elif func == 'TF-IDF':
        retrieved_docs = tf_idf_topk(question,all_docs,k)
      elif func == 'BM25':
        retrieved_docs = bm25_topk(question,all_docs,k)

      predictions.append({'question':question,'predictions':retrieved_docs})

      found_relevant_doc = False
      relevant_docs_found = 0
      current_avg_precision = 0.0

      for doc_idx, doc in enumerate(retrieved_docs):
        for gold_id in qdict['dids']:
          if str(doc) == gold_id:
            relevant_docs_found += 1
            if not found_relevant_doc:
                correct_retrievals += 1
                summed_reciprocal_rank += 1 / (doc_idx + 1)
            current_avg_precision += relevant_docs_found / (doc_idx + 1)
            found_relevant_doc = True
            break

      if found_relevant_doc:
          all_relevant_docs = qdict['n_docs']
          summed_avg_precision += current_avg_precision / all_relevant_docs

      avg_f1 += compute_f1(qdict['dids'],retrieved_docs)
      
    avg_f1 = avg_f1 / number_of_questions
    recall = correct_retrievals / number_of_questions
    mean_reciprocal_rank = summed_reciprocal_rank / number_of_questions
    mean_avg_precision = summed_avg_precision / number_of_questions

    metrics =  {
        "recall": recall,
        "map": mean_avg_precision,
        "mrr": mean_reciprocal_rank,
        "f1":avg_f1,
        "n_questions": number_of_questions,
        "n_docs": len(all_docs),
        "top_k": k
    }
    all_metrics.append(metrics)

  return all_metrics, predictions

In [16]:
def plot(all_metrics,func):
  rs, ps, fs, rrs, aavg, ks = [], [], [], [], [], []
  for m in all_metrics:
    rs.append(m['recall'])
    ps.append(m['map'])
    rrs.append(m['mrr'])
    fs.append(m['f1'])
    ks.append(m['top_k'])

    tot = np.sum([rs,fs,ps,rrs])
    aavg.append(np.divide(tot, 4))

  pyplot.plot(ks,ps,label='MAP')
  pyplot.plot(ks,rs,label='Recall')
  pyplot.plot(ks,rrs,label='MRR')
  pyplot.plot(ks,fs,label='F1')
  pyplot.plot(ks,np.divide(aavg, len(ks)),label='AAVG')
  pyplot.title(func)
  pyplot.legend()

### Tests

1. Plots
    * File: MLQA_ALL_1k
    * Models: LaDPR, DPR, BM25, TF-IDF
    * top-k: [1,3,5,7,10,15,20,25,30,40,50,75,100]
2. SQAL
    * File: MLQA_X_X (all seven languages)
    * Models: LaDPR, DPR, BM25, TF-IDF
    * top-k: 20
3. ALL
    * MLQA_ALL
    * Models: LaDPR, DPR, BM25, TF-IDF
    * top-k: 20
    
    



In [17]:
def run_eval():
    top_ks = [20,50,100]#,50]
    #top_ks = [1,3,5,7,10,15,20,25,30,40,50,60,70,80,90,100]
    
    funcs = ['LaDPR']#,'BM25','TF-IDF']
    for func in funcs:
        for k in top_ks:
            all_metrics, predictions = eval([k],func)
            #plot(all_metrics,func)
            print(all_metrics,'\n')
            df = pd.DataFrame(all_metrics)
            print(df,'\n\n')
            csv_name = file_name + '__' + str(k) + '__' + func + '.csv'
            df.to_csv(csv_name)

In [18]:
run_eval()

100%|██████████| 39015/39015 [41:26<00:00, 15.69it/s]


[{'recall': 0.4161988978597975, 'map': 0.07232993111521985, 'mrr': 0.20711275379076796, 'f1': 0.05732145025693502, 'n_questions': 39015, 'n_docs': 33915, 'top_k': 20}] 

     recall      map       mrr        f1  n_questions  n_docs  top_k
0  0.416199  0.07233  0.207113  0.057321        39015   33915     20 




100%|██████████| 39015/39015 [45:51<00:00, 14.18it/s]


[{'recall': 0.50959887222863, 'map': 0.07682889870105683, 'mrr': 0.2101161473708758, 'f1': 0.0415104484322617, 'n_questions': 39015, 'n_docs': 33915, 'top_k': 50}] 

     recall       map       mrr       f1  n_questions  n_docs  top_k
0  0.509599  0.076829  0.210116  0.04151        39015   33915     50 




100%|██████████| 39015/39015 [51:56<00:00, 12.52it/s]


[{'recall': 0.5808278867102397, 'map': 0.07890136236585539, 'mrr': 0.21113699673619107, 'f1': 0.03012834633752614, 'n_questions': 39015, 'n_docs': 33915, 'top_k': 100}] 

     recall       map       mrr        f1  n_questions  n_docs  top_k
0  0.580828  0.078901  0.211137  0.030128        39015   33915    100 




In [19]:
# clearing output
def clear_all():
    import os
    for f in glob('/kaggle/working/*.csv'):    
        os.remove(f)

#clear_all()